<a href="https://colab.research.google.com/github/Chevron9/iannwtf/blob/Roho7/10/Exercise10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import random
%load_ext tensorboard
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import datetime
import copy
%matplotlib notebook

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [82]:
# Load dataset, split into test and training data
train_ds = tfds.load('tiny_shakespeare', split= 'train', data_dir="data\\", shuffle_files=True)

for elem in train_ds.take(1):
  print(elem)

{'text': <tf.Tensor: shape=(), dtype=string, numpy=b"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we beco

In [83]:
#extract the text
train_np = tfds.as_numpy(train_ds)


text = ""
for elem in train_np:
   text += {key: elem[key] for key in elem.keys() 
                                & {"text"}}["text"].decode("utf-8")


#format the text
text = text.lower()
text = text.replace('\n', '')
punctuation = '''!/?@#$%^&*_~()-[]{};:'"\,<>.'''
final_text = ""
for character in text:
  if character not in punctuation:
    final_text = final_text + character
  if character in punctuation:
    final_text = final_text + " "

#list of all words in order
word_list = final_text.split() 

#set of all words
word_set = set(word_list)


#creating the dictionaries
idtostring = dict() 
key = 0
for ele in word_set: 
    idtostring[str(key)] = ele
    key +=1

stringtoid = dict((v,k) for k,v in idtostring.items())

print(final_text)
print(text)
print(len(word_list))
print(word_set)
print(idtostring)
print(stringtoid)


first citizen before we proceed any further  hear me speak all speak  speak first citizen you are all resolved rather to die than to famish all resolved  resolved first citizen first  you know caius marcius is chief enemy to the people all we know t  we know t first citizen let us kill him  and we ll have corn at our own price is t a verdict all no more talking on t  let it be done  away  away second citizen one word  good citizens first citizen we are accounted poor citizens  the patricians good what authority surfeits on would relieve us  if theywould yield us but the superfluity  while it werewholesome  we might guess they relieved us humanely but they think we are too dear  the leanness thatafflicts us  the object of our misery  is as aninventory to particularise their abundance  oursufferance is a gain to them let us revenge this withour pikes  ere we become rakes  for the gods know ispeak this in hunger for bread  not in thirst for revenge second citizen would you proceed especia

In [84]:
#generates random input-context pairs in one-hot encoding
list_len =len(word_list)
def pairs():
  while True:
      
    random_number = random.randint(0, list_len)
    input = [tf.one_hot(int(stringtoid[word_list[random_number]]), tf.size(word_list), dtype=tf.float32)]
    context = [tf.one_hot(int(stringtoid[word_list[random_number-2]]), tf.size(word_list), dtype=tf.float32),
               tf.one_hot(int(stringtoid[word_list[random_number-1]]), tf.size(word_list), dtype=tf.float32),
               tf.one_hot(int(stringtoid[word_list[random_number+1]]), tf.size(word_list), dtype=tf.float32)]
    

    #goes over all pairs for one word
    for i in range(len(context)):
      yield tf.reshape(input, (1,-1)), tf.reshape((context[i]), (1,-1))

In [85]:
#creating dataset from pairs
dataset_gen = tf.data.Dataset.from_generator(pairs,
                                             output_signature=(
                                              tf.TensorSpec(shape=(1, tf.size(word_list)), dtype=tf.float32),
                                              tf.TensorSpec(shape=(1, tf.size(word_list)), dtype=tf.float32)))


for elem in dataset_gen.take(1):
  print("Input", elem[0])
  print("Target", elem[1])
   

Input tf.Tensor([[0. 0. 0. ... 0. 0. 0.]], shape=(1, 181699), dtype=float32)
Target tf.Tensor([[0. 0. 0. ... 0. 0. 0.]], shape=(1, 181699), dtype=float32)


In [86]:
# the embedding
class SkipGram(tf.keras.layers.Layer):
  def __init__(self):
    super(SkipGram, self).__init__()
